## Retrieving OPERA disturbance data

The [OPERA DIST-HLS data product](https://lpdaac.usgs.gov/documents/1766/OPERA_DIST_HLS_Product_Specification_V1.pdf) can be used to study vegetation disturbances over time. In this notebook, we will retrieve disturbance data 
over
associated with the [2023 Greece wildfires](https://en.wikipedia.org/wiki/2023_Greece_wildfires) to understand its evolution and extent. We will also generate a time series visualization of the event

In [ ]:
import rasterio
import rioxarray
import hvplot.pandas  # noqa
import hvplot.xarray  # noqa
from rasterio.warp import transform_bounds, reproject
from rasterio.crs import CRS
import contextily as cx
import matplotlib.pyplot as plt

import xarray as xr
import xyzservices.providers as xyz

from shapely.geometry import Point
from osgeo import gdal


import pandas as pd

# Imports for plotting
import geoviews as gv
gv.extension('bokeh')
gv.output(size=1000)

# STAC imports to retrieve cloud data
from pystac_client import Client

from datetime import datetime
import numpy as np

# GDAL setup for accessing cloud data
gdal.SetConfigOption('GDAL_HTTP_COOKIEFILE','~/cookies.txt')
gdal.SetConfigOption('GDAL_HTTP_COOKIEJAR', '~/cookies.txt')
gdal.SetConfigOption('GDAL_DISABLE_READDIR_ON_OPEN','EMPTY_DIR')
gdal.SetConfigOption('CPL_VSIL_CURL_ALLOWED_EXTENSIONS','TIF, TIFF')


In [ ]:
# Let's set up the parameters of our search query

# Flooding in Texas, 2024; 
wadi_sirhan_basin = Point(16.5, -16.09).buffer(0.01)

# We will search data around the flooding event at the start of May
start_date = datetime(year=2023, month=1, day=1)
stop_date = datetime(year=2024, month=6, day=1)
date_range = f'{start_date.strftime("%Y-%m-%d")}/{stop_date.strftime("%Y-%m-%d")}'

# We open a client instance to search for data, and retrieve relevant data records
STAC_URL = 'https://cmr.earthdata.nasa.gov/stac'

# Setup PySTAC client
# LPCLOUD refers to the LP DAAC cloud environment that hosts OPERA DIST data

catalog = Client.open(f'{STAC_URL}/LPCLOUD/')
collections = ["OPERA_L3_DIST-ALERT-HLS_V1"]

opts = {
    'bbox' : wadi_sirhan_basin.bounds, 
    'collections': collections,
    'datetime' : date_range,
}

In [ ]:
# Execute the search
search = catalog.search(**opts)
results = list(search.items_as_dicts())
print(f"Number of tiles found intersecting given AOI: {len(results)}")

In [ ]:
layer_name = 'VEG-DIST-STATUS'

times = pd.DatetimeIndex([result['properties']['datetime'] for result in results]) # parse of timestamp for each result
data = {'hrefs': [value['href'] for result in results for key, value in result['assets'].items() if layer_name in key],  # parse out links only to DIST-STATUS data layer
        'tile_id': [value['href'].split('/')[-1].split('_')[3] for result in results for key, value in result['assets'].items() if layer_name in key]}


In [ ]:
# # Construct pandas dataframe to summarize granules from search results
granules = pd.DataFrame(index=times, data=data)
granules.index.name = 'times'
granules.head()

In [ ]:
test_granules = granules[:10:]

In [ ]:
len(test_granules)

In [ ]:
def urls_to_dataset(granule_dataframe):
    '''method that takes in a list of OPERA tile URLs and returns an xarray dataset with dimensions
    latitude, longitude and time'''

    dataset_list = []
    
    for i, row in granule_dataframe.iterrows():
        with rasterio.open(row.hrefs) as ds:
            # extract CRS string
            crs = str(ds.crs).split(':')[-1]

            # extract the image spatial extent (xmin, ymin, xmax, ymax)
            xmin, ymin, xmax, ymax = ds.bounds

            # the x and y resolution of the image is available in image metadata
            x_res = np.abs(ds.transform[0])
            y_res = np.abs(ds.transform[4])

            # read the data 
            img = ds.read()

            # Ensure img has three dimensions (bands, y, x)
            if img.ndim == 2:
                img = np.expand_dims(img, axis=0) 

            

        lon = np.arange(xmin, xmax, x_res)
        lat = np.arange(ymax, ymin, -y_res)

        lon_grid, lat_grid = np.meshgrid(lon, lat)

        da = xr.DataArray(
            data=img,
            dims=["band", "y", "x"],
            coords=dict(
                lon=(["y", "x"], lon_grid),
                lat=(["y", "x"], lat_grid),
                time=i,
                band=np.arange(img.shape[0])
            ),
            attrs=dict(
                description="OPERA DIST",
                units=None,
            ),
        )
        da.rio.write_crs(crs, inplace=True)

        dataset_list.append(da)
    return xr.concat(dataset_list, dim='time').squeeze()

dataset= urls_to_dataset(test_granules)

In [ ]:
COLORS = [(0, 0, 0, 0)]*256 # Set cmap where all classes are zero opacity
COLORS[1] = (255, 0, 0, 1)
COLORS[2] = (255, 0, 0, 1)
COLORS[4] = (255, 0, 0, 1) # We only visualize confirmed disturbances

In [ ]:
img = dataset.hvplot.quadmesh(title = 'Crop circles in UAE',
                            x='lon', y='lat', 
                            project=True, rasterize=True, frame_width=100,
                            cmap=COLORS, 
                            colorbar=False,
                            widget_location='bottom',
                            tiles = xyz.OpenStreetMap.Mapnik,
                            xlabel='Longitude (degrees)',ylabel='Latitude (degrees)',
                            fontscale=1.25)

img

In [ ]:
import geopandas as gpd

In [ ]:
gdf = gpd.read_file('/home/karthik/Desktop/workspace/pydata-seattle-2024/data/landsat_ot_c2_l2_667298b52c41d8f6/landsat_ot_c2_l2_667298b52c41d8f6.shp')

In [ ]:
gdf.hvplot(geo=True)

In [ ]:
final_geometry = gdf['geometry'].unary_union

In [ ]:
test_gdf = gpd.GeoDataFrame(data = {'merged_shape':[0]}, geometry=[final_geometry])

In [ ]:
test_gdf

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
test_gdf.geometry.plot(ax=ax, facecolor="none", edgecolor="red", linewidth=1)
cx.add_basemap(ax, crs=gdf.crs, zoom=1, source=cx.providers.OpenStreetMap.Mapnik, attribution='')